In [1]:
# import dependencies
import pandas as pd
import numpy as np
import os

from time import strftime
from time import time

import warnings
warnings.filterwarnings("ignore")

pathway = os.path.abspath('')
pathway_data = os.path.join(pathway, "Subway_Bus_Streetcar")
pathway_folder = os.path.join(pathway_data, "Data")
print(f"Pathway to Folder Holding Raw and Processed Data: ${pathway_folder}")

Pathway to Folder Holding Raw and Processed Data: $C:\Users\danie\OneDrive\Documents\fix_TTC_Delays\TTC_Delays\Jupyter-Notebooks\Subway_Bus_Streetcar\Data


# Subway Cleanup

In [2]:
def sheetsTTC(xlsx_file):
    
    """ 
    Reads in an Excel file and concatenates each sheet into 
    a returned pandas DataFrame.    
    
    Note:
        - Requires: pandas
        - Requires: Each worksheet must have exact matching
                    n columns, column order, column labels
                    and column dtypes.
    """
    
    xls = pd.ExcelFile(xlsx_file)
    
    out_df = pd.DataFrame()
    
    for sheet in xls.sheet_names:
        
        df = pd.read_excel(xls, sheet_name = sheet)
        
        df = df[[
            #'Report Date', 'Route', 'Time', 'Day', 'Location', 'Incident', 'Min Delay', 'Min Gap', 'Direction', 'Vehicle'
            'Date', 'Time', 'Day', 'Station', 'Code', 'Min Delay', 'Min Gap', 'Bound', 'Line', 'Vehicle'
        ]]
        
        out_df = pd.concat([out_df, df])
        
    return out_df

def excelDate(excel_time):
    
    """
    Converts excel datetime float format to pandas datetime
    
    """
    
    return pd.to_datetime('1900-01-01') + pd.to_timedelta(excel_time, 'D')


### Import Subway Files, Merge files

In [3]:
sub19 = sheetsTTC(os.path.join(pathway_folder, "Subway_2019.xlsx"))
sub18 = sheetsTTC(os.path.join(pathway_folder, "Subway_2018.xlsx"))
sub17 = sheetsTTC(os.path.join(pathway_folder, "Subway_2017.xlsx"))
sub1417 = sheetsTTC(os.path.join(pathway_folder, "Subway_2014_042017.xlsx"))

sub19['Date'] = excelDate(sub19['Date'])
sub18['Date'] = excelDate(sub18['Date'])
sub17['Date'] = excelDate(sub17['Date'])
sub1417['Date'] = excelDate(sub1417['Date'])

sub19['Year'] = 2019
sub18['Year'] = 2018
sub17['Year'] = 2017
sub1417['Year']  = sub1417['Date'].dt.year

subway = pd.concat([sub1417, sub17])
subway = pd.concat([subway, sub18])
subway = pd.concat([subway, sub19])

codes = pd.read_csv(os.path.join(pathway_folder, 'Log_Codes.csv'))
subway = pd.merge(subway, codes, on='Code', how='left')

pathway_clean = os.path.join(pathway_data, "Clean_Data")
subway.to_csv(os.path.join(pathway_clean, 'ttc_subway_srt.csv'))

In [4]:
len(subway)

109326

In [5]:
subway.sample(5)

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,Year,Description
34,2014-01-03,18:07,Wednesday,KENNEDY BD STATION,MUIR,5,10,W,BD,5159,2014,Injured or ill Customer (On Train) - Medical A...
4339,2014-03-13,08:26,Tuesday,VICTORIA PARK STATION,MUPAA,0,0,W,BD,5886,2014,Passenger Assistance Alarm Activated - No Trou...
48389,2016-04-29,15:48,Wednesday,ST GEORGE YUS STATION,TUSC,0,0,S,YU,5996,2016,Operator Overspeeding
27637,2015-05-03,06:42,Friday,KIPLING STATION,MUSC,0,0,E,BD,5068,2015,Miscellaneous Speed Control
16919,2014-10-30,22:41,Tuesday,GREENWOOD YARD,MUWR,0,0,NaN,BD,0,2014,Work Refusal


# Add weather, holidays and loactions to merged Subway Dataframe

In [6]:
subway = pd.read_csv(os.path.join(pathway_clean, 'ttc_subway_srt.csv'))
weather = pd.read_csv(os.path.join(pathway, 'Toronto_Weather', 'Merged_Data', 'Completely_Averaged.csv'))
holidays = pd.read_csv(os.path.join(pathway_folder, 'holidays.csv'))
locations = pd.read_csv(os.path.join(pathway_folder, 'subway_location.csv'))

In [7]:
#subway.sample(10)

In [8]:
subway['Date'] = pd.to_datetime(subway['Date'])
weather['Date/Time'] = pd.to_datetime(weather['Date/Time'])
holidays['date'] = pd.to_datetime(holidays['date'])

weather = weather.rename(columns={
    'Date/Time': 'Date', 
    'Mean Temp (°C)': 'Mean Temp', 
    'Total Precip (mm)': 'Precipitation'
})

holidays = holidays.rename(columns={
    'date': 'Date'
})

subway['Month'] = subway['Date'].dt.month
subway['nDay'] = subway['Date'].dt.day

weather = weather.drop(columns=['Year', 'Month', 'Day'])

subway = pd.merge(subway, weather, on='Date', how='left')
subway = pd.merge(subway, holidays, on='Date', how='left')

subway['holiday'] = subway['holiday'].fillna('None')

subway['Line'] = subway['Line'].replace({
    'B/D': 'BD', 
    'YU/ BD': 'YU/BD', 
    'YU / BD': 'YU/BD', 
    'BD LINE': 'BD', 
    'BD/YU': 'YU/BD', 
    'BD/YUS': 'YU/BD', 
    'YUS': 'YU', 
    'YU-BD': 'YU/BD', 
    'YU & BD': 'YU/BD', 
    'YU - BD LINE': 'YU/BD', 
    'YU BD': 'YU/BD',
    'YU LINE': 'YU',
    'BLOOR DANFORTH': 'BD',
    'YONGE UNIVERSITY SERVI': 'YU',
    'SHEPPARD': 'SHP',
    '25 DON MILLS': 'SHP',
    'YUBD': 'YU/BD',
    'YU - BD': 'YU/BD',
    '66': 'YU/BD'
})

subway['Precipitation'] = subway['Total Rain (mm)'] + (subway['Total Snow (cm)']*10)

In [9]:
subway.columns

Index(['Unnamed: 0', 'Date', 'Time', 'Day', 'Station', 'Code', 'Min Delay',
       'Min Gap', 'Bound', 'Line', 'Vehicle', 'Year', 'Description', 'Month',
       'nDay', 'Max Temp (°C)', 'Min Temp (°C)', 'Mean Temp',
       'Total Rain (mm)', 'Total Snow (cm)', 'Precipitation',
       'Snow on Grnd (cm)', 'Spd of Max Gust (km/h)', 'holiday'],
      dtype='object')

In [10]:
subway.drop(['Unnamed: 0'], axis=1, inplace=True)
subway.head()

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,...,nDay,Max Temp (°C),Min Temp (°C),Mean Temp,Total Rain (mm),Total Snow (cm),Precipitation,Snow on Grnd (cm),Spd of Max Gust (km/h),holiday
0,2014-01-03,00:21,Wednesday,VICTORIA PARK STATION,MUPR1,55,60,W,BD,5111,...,3,-7.6,-23.6,-15.6,0.0,0.0,0.0,7.6,34.0,None
1,2014-01-03,02:06,Wednesday,HIGH PARK STATION,SUDP,3,7,W,BD,5001,...,3,-7.6,-23.6,-15.6,0.0,0.0,0.0,7.6,34.0,None
2,2014-01-03,02:40,Wednesday,SHEPPARD STATION,MUNCA,0,0,NaN,YU,0,...,3,-7.6,-23.6,-15.6,0.0,0.0,0.0,7.6,34.0,None
3,2014-01-03,03:10,Wednesday,LANSDOWNE STATION,SUDP,3,8,W,BD,5116,...,3,-7.6,-23.6,-15.6,0.0,0.0,0.0,7.6,34.0,None
4,2014-01-03,03:20,Wednesday,BLOOR STATION,MUSAN,5,10,S,YU,5386,...,3,-7.6,-23.6,-15.6,0.0,0.0,0.0,7.6,34.0,None


In [11]:
subway.to_csv(os.path.join(pathway_clean, 'subway_master.csv'), index=False)

In [12]:
subway.sample(5)

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,...,nDay,Max Temp (°C),Min Temp (°C),Mean Temp,Total Rain (mm),Total Snow (cm),Precipitation,Snow on Grnd (cm),Spd of Max Gust (km/h),holiday
108828,2019-04-22,19:51,Saturday,ST CLAIR WEST STATION,PUMEL,0,0,NaN,YU,0,...,22,16.5,4.7,10.6,0.0,0.0,0.0,0.0,0.0,None
108855,2019-04-23,14:26,Sunday,SHEPPARD WEST STATION,MUATC,0,0,S,YU,5446,...,23,14.3,6.5,10.4,1.7,0.0,1.7,0.0,56.0,None
3776,2014-03-05,05:38,Monday,DON MILLS STATION,TUSC,3,8,W,SHP,5248,...,5,-5.1,-14.0,-9.6,0.0,0.0,0.0,19.2,31.0,None
56795,2016-09-19,09:38,Saturday,MUSEUM STATION,MUSC,0,0,S,YU,5911,...,19,26.8,15.1,21.0,0.0,0.0,0.0,0.0,21.0,None
28772,2015-05-22,10:08,Wednesday,ST ANDREW STATION,MUPAA,0,0,N,YU,5896,...,22,15.3,5.2,10.3,0.0,0.0,0.0,0.0,54.0,None


# Surface Routes (Buses and Streetcars) Cleaning

## Merging all the raw Excel files downloaded

### Buses

In [13]:
def sheetsTTCSurface(xlsx_file):
    
    """ 
    Reads in an Excel file and concatenates each sheet into 
    a returned pandas DataFrame.    
    
    Note:
        - Requires: pandas
        - Requires: Each worksheet must have exact matching
                    n columns, column order, column labels
                    and column dtypes.
    """
    
    xls = pd.ExcelFile(xlsx_file)
    
    out_df = pd.DataFrame()
    
    for sheet in xls.sheet_names:
        
        df = pd.read_excel(xls, sheet_name = sheet)
        
        df = df[[
            'Report Date', 'Route', 'Time', 'Day', 'Location', 'Incident', 'Min Delay', 'Min Gap', 'Direction', 'Vehicle'
        ]]
        
        out_df = pd.concat([out_df, df])
        
    return out_df

In [14]:
# Bus 2019
bus2019_xls = pd.ExcelFile(os.path.join(pathway_folder, 'Bus_2019.xlsx'))

jan = pd.read_excel(bus2019_xls, 'Jan 2019')
feb = pd.read_excel(bus2019_xls, 'Feb 2019')
mar = pd.read_excel(bus2019_xls, 'Mar 2019')
apr = pd.read_excel(bus2019_xls, 'Apr 2019')
may = pd.read_excel(bus2019_xls, 'May 2019')

# April is the only month in the 5 1/2 years of data
# with the column 'Incident ID', so it gets dropped
apr = apr.drop(columns={'Incident ID'})

# Rename the two mis-labelled columns
apr = apr.rename(columns={'Delay': 'Min Delay', 'Gap': 'Min Gap'})

bus19 = pd.concat([jan, feb])
bus19 = pd.concat([bus19, mar])
bus19 = pd.concat([bus19, apr])
bus19 = pd.concat([bus19, may])

# Apply sheetsTTC Function
bus18 = sheetsTTCSurface(os.path.join(pathway_folder, 'Bus_2018.xlsx'))
bus17 = sheetsTTCSurface(os.path.join(pathway_folder, 'Bus_2017.xlsx'))
bus16 = sheetsTTCSurface(os.path.join(pathway_folder, 'Bus_2016.xlsx'))
bus15 = sheetsTTCSurface(os.path.join(pathway_folder, 'Bus_2015.xlsx'))
bus14 = sheetsTTCSurface(os.path.join(pathway_folder, 'Bus_2014.xlsx'))

# Add Year Columns
bus19['Year'] = 2019
bus18['Year'] = 2018
bus17['Year'] = 2017
bus16['Year'] = 2016
bus15['Year'] = 2015
bus14['Year'] = 2014

# Combine 2014 - 2019
buses = pd.concat([bus19, bus18])
buses = pd.concat([buses, bus17])
buses = pd.concat([buses, bus16])
buses = pd.concat([buses, bus15])
buses = pd.concat([buses, bus14])

# Add Bus labeled Type column
buses['Type'] = 'Bus'

In [15]:
buses.head()

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Year,Type
0,43466,39,0.009028,Tuesday,NECR,Mechanical,9.0,18.0,W/B,1794.0,2019,Bus
1,43466,111,0.010417,Tuesday,Eglington,Mechanical,15.0,30.0,S/B,8065.0,2019,Bus
2,43466,35,0.012500,Tuesday,Finch,Mechanical,9.0,18.0,S/B,3275.0,2019,Bus
3,43466,25,0.020833,Tuesday,Don Mills Rd/Eglinton Ave E,Mechanical,9.0,18.0,N/B,8840.0,2019,Bus
4,43466,36,0.027778,Tuesday,Humberwood,Investigation,9.0,18.0,E/B,9119.0,2019,Bus


### Streetcar

In [16]:
# Apply sheetsTTC Function
streetcar19 = sheetsTTCSurface(os.path.join(pathway_folder, 'Streetcar_2019.xlsx'))
streetcar18 = sheetsTTCSurface(os.path.join(pathway_folder, 'Streetcar_2018.xlsx'))
streetcar17 = sheetsTTCSurface(os.path.join(pathway_folder, 'Streetcar_2017.xlsx'))
streetcar16 = sheetsTTCSurface(os.path.join(pathway_folder, 'Streetcar_2016.xlsx'))
streetcar15 = sheetsTTCSurface(os.path.join(pathway_folder, 'Streetcar_2015.xlsx'))
streetcar14 = sheetsTTCSurface(os.path.join(pathway_folder, 'Streetcar_2014.xlsx'))

# Add Year Columns
streetcar19['Year'] = 2019
streetcar18['Year'] = 2018
streetcar17['Year'] = 2017
streetcar16['Year'] = 2016
streetcar15['Year'] = 2015
streetcar14['Year'] = 2014

# Combine 2014 - 2019
streetcars = pd.concat([streetcar19, streetcar18])
streetcars = pd.concat([streetcars, streetcar17])
streetcars = pd.concat([streetcars,streetcar16])
streetcars = pd.concat([streetcars, streetcar15])
streetcars = pd.concat([streetcars, streetcar14])

# Add Streetcar labeled Type column
streetcars['Type'] = 'Streetcar'

In [17]:
streetcars.head()

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Year,Type
0,43466,301,0.047222,Tuesday,Queen/Braodview,Held By,6.0,13.0,E/B,4193.0,2019,Streetcar
1,43466,511,0.062500,Tuesday,Bathurst/College,Investigation,5.0,10.0,N/B,1038.0,2019,Streetcar
2,43466,306,0.069444,Tuesday,Dundas West stn.,Mechanical,8.0,16.0,W/B,4146.0,2019,Streetcar
3,43466,505,0.131944,Tuesday,Lansdowne and Dundas,Mechanical,6.0,12.0,E/B,8416.0,2019,Streetcar
4,43466,310,0.163194,Tuesday,Spadina and Lakshore,Held By,20.0,30.0,N/B,4465.0,2019,Streetcar


### Surface Routes (Streetcar + Buses) 

In [18]:
#Merge streetcar and buses dataframes to a single dataframe
surface = pd.concat([buses, streetcars])

In [19]:
surface.head()

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Year,Type
0,43466,39,0.009028,Tuesday,NECR,Mechanical,9.0,18.0,W/B,1794.0,2019,Bus
1,43466,111,0.010417,Tuesday,Eglington,Mechanical,15.0,30.0,S/B,8065.0,2019,Bus
2,43466,35,0.012500,Tuesday,Finch,Mechanical,9.0,18.0,S/B,3275.0,2019,Bus
3,43466,25,0.020833,Tuesday,Don Mills Rd/Eglinton Ave E,Mechanical,9.0,18.0,N/B,8840.0,2019,Bus
4,43466,36,0.027778,Tuesday,Humberwood,Investigation,9.0,18.0,E/B,9119.0,2019,Bus


In [20]:
surface['Report Date'] = excelDate(surface['Report Date'])
surface['Time'] = round(((surface['Time'] * 86400)/60)/60, 2)

In [21]:
surface.head()

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Year,Type
0,2019-01-03,39,0.22,Tuesday,NECR,Mechanical,9.0,18.0,W/B,1794.0,2019,Bus
1,2019-01-03,111,0.25,Tuesday,Eglington,Mechanical,15.0,30.0,S/B,8065.0,2019,Bus
2,2019-01-03,35,0.30,Tuesday,Finch,Mechanical,9.0,18.0,S/B,3275.0,2019,Bus
3,2019-01-03,25,0.50,Tuesday,Don Mills Rd/Eglinton Ave E,Mechanical,9.0,18.0,N/B,8840.0,2019,Bus
4,2019-01-03,36,0.67,Tuesday,Humberwood,Investigation,9.0,18.0,E/B,9119.0,2019,Bus


In [22]:
surface.columns

Index(['Report Date', 'Route', 'Time', 'Day', 'Location', 'Incident',
       'Min Delay', 'Min Gap', 'Direction', 'Vehicle', 'Year', 'Type'],
      dtype='object')

In [23]:
surface.to_csv(os.path.join(pathway_clean, 'ttc_surface_route_delays.csv'))
surface.sample(25)

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Year,Type
5287,2014-05-24,67,14.50,Thursday,67 Pharmacy Route,General Delay,5.0,15.0,BW,NaN,2014,Bus
5594,2019-04-02,900,0.92,Sunday,Kipling stn,Utilized Off Route,10.0,20.0,N/B,8003.0,2019,Bus
2907,2018-11-17,117,19.37,Thursday,Gerry Fitzgerald and Steeles,Investigation,12.0,24.0,N/B,1247.0,2018,Bus
1040,2016-11-06,57,18.23,Friday,Midland & Steeles,Mechanical,8.0,16.0,SB,7836.0,2016,Bus
1446,2017-01-12,69,6.25,Tuesday,Sadler and danforth,Mechanical,6.0,12.0,sb,7806.0,2017,Bus
6084,2019-02-28,927,12.03,Tuesday,NaN,Investigation,6.0,12.0,NaN,3211.0,2019,Bus
7048,2016-11-02,96,13.08,Monday,Wilson stn,Utilized Off Route,10.0,20.0,e,1414.0,2016,Bus
5630,2014-08-29,37,6.37,Wednesday,Steeles and Islington,Mechanical,9.0,18.0,S,NaN,2014,Bus
8442,2014-01-30,54,7.80,Tuesday,Eglinton stn,Mechanical,6.0,12.0,eb,7453.0,2014,Bus
4396,2015-07-25,801,16.43,Thursday,Wislon Division,Utilized Off Route,1.0,1.0,N,9430.0,2015,Bus


# Add weather, holidays and loactions to merged Surface Routes (Bus and Streetcar) Dataframe

In [24]:
#Import cleaned up CSV files
surface = pd.read_csv(os.path.join(pathway_clean, 'ttc_surface_route_delays.csv'))
weather = pd.read_csv(os.path.join(pathway, 'Toronto_Weather', 'Merged_Data', 'Completely_Averaged.csv'))
holidays = pd.read_csv(os.path.join(pathway_folder, 'holidays.csv'))

In [25]:
#Merging Surface data, weather and holidaya
surface['Report Date'] = pd.to_datetime(surface['Report Date'])
surface['Month'] = surface['Report Date'].dt.month
surface['nDay'] = surface['Report Date'].dt.day
surface = surface.rename(columns={'Report Date': 'Date'})
surface = surface.drop(columns=['Unnamed: 0'])

weather['Date/Time'] = pd.to_datetime(weather['Date/Time'])
weather = weather.rename(columns={
    'Date/Time': 'Date', 
    'Mean Temp (°C)': 'Mean Temp', 
    'Total Precip (mm)': 'Precipitation'
})
weather = weather.drop(columns={'Year', 'Month', 'Day'})

holidays = holidays.rename(columns={'date': 'Date', 'holiday': 'Holiday'})
holidays['Date'] = pd.to_datetime(holidays['Date'])

surface = pd.merge(surface, weather, on ='Date', how='left')
surface = pd.merge(surface, holidays, on ='Date', how='left')

surface['Holiday'] = surface['Holiday'].fillna('None')

surface = surface[surface['Min Delay'] < 7000]
surface['Route'] = surface['Route'].astype('category')
surface['Vehicle'] = surface['Vehicle'].astype('category')

surface['Incident'] = surface['Incident'].replace({
    'Late Leaving Garage - Mechanical': 'Late Leaving Garage',
    'Late Leaving Garage - Operator': 'Late Leaving Garage',
    'Late Leaving Garage - Management': 'Late Leaving Garage'
})

surface = surface[(surface['Min Delay'] >= 0) & (surface['Min Delay'] <= 30)]

In [26]:
surface.columns

Index(['Date', 'Route', 'Time', 'Day', 'Location', 'Incident', 'Min Delay',
       'Min Gap', 'Direction', 'Vehicle', 'Year', 'Type', 'Month', 'nDay',
       'Max Temp (°C)', 'Min Temp (°C)', 'Mean Temp', 'Total Rain (mm)',
       'Total Snow (cm)', 'Precipitation', 'Snow on Grnd (cm)',
       'Spd of Max Gust (km/h)', 'Holiday'],
      dtype='object')

In [27]:
#Export merged data
surface.to_csv(os.path.join(pathway_clean, 'surface_master.csv'), index=False)

In [28]:
surface.head()

,Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,...,nDay,Max Temp (°C),Min Temp (°C),Mean Temp,Total Rain (mm),Total Snow (cm),Precipitation,Snow on Grnd (cm),Spd of Max Gust (km/h),Holiday
0,2019-01-03,39,0.22,Tuesday,NECR,Mechanical,9.0,18.0,W/B,1794.0,...,3,1.3,-2.5,-0.6,0.0,0.0,0.0,2.8,44.0,None
1,2019-01-03,111,0.25,Tuesday,Eglington,Mechanical,15.0,30.0,S/B,8065.0,...,3,1.3,-2.5,-0.6,0.0,0.0,0.0,2.8,44.0,None
2,2019-01-03,35,0.30,Tuesday,Finch,Mechanical,9.0,18.0,S/B,3275.0,...,3,1.3,-2.5,-0.6,0.0,0.0,0.0,2.8,44.0,None
3,2019-01-03,25,0.50,Tuesday,Don Mills Rd/Eglinton Ave E,Mechanical,9.0,18.0,N/B,8840.0,...,3,1.3,-2.5,-0.6,0.0,0.0,0.0,2.8,44.0,None
4,2019-01-03,36,0.67,Tuesday,Humberwood,Investigation,9.0,18.0,E/B,9119.0,...,3,1.3,-2.5,-0.6,0.0,0.0,0.0,2.8,44.0,None
